# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key

%load_ext autoreload
%autoreload 2

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# create a reference to the csv file
cities_csv = "output_data/cities.csv"
# read the csv file into a dataframe
cities_df = pd.read_csv(cities_csv)
cities_df = cities_df.dropna()
del cities_df["Unnamed: 0"]
cities_df.head()
cities_df.count()

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' as locations
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)



In [ ]:
# Add heatmap layer to map
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius = 5)

fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition.
# Drop any rows that don't contain all three conditions

# A max temperature lower than 80 degrees but higher than 70.
city_low_temp = cities_df[cities_df["Max Temp"] > 70] 
city_max_temp = city_low_temp[city_low_temp["Max Temp"] < 80]
# Zero cloudiness
city_cloudless = city_max_temp[city_max_temp["Cloudiness"] == 0]
# Wind speed less than 10 mph
city_wind = city_cloudless[city_cloudless["Wind Speed"] < 40]
# Store into hotel variable name
hotel_df = city_wind
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
reduced_cities_df = cities_df.loc[(cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80)].dropna()
reduced_cities_df.count()

In [ ]:
hotel_df = city_wind.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df

In [ ]:
import json
import pprint
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    pprint.pprint(response)
    # extract results
    results = response['results']
    print(json.dumps(response, indent=4, sort_keys=True))
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
    print("------------")
    

In [ ]:
# Add "Hotel Name" column to DF
hotel_df["Hotel Name"] = "" 
hotel_name = []

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    
# Set parameters
target_radius = 5000
target_type = "hotel"
params = {
    "key": g_key,
    "radius": target_radius,
    "type": target_type
    }

    # Loop through the hotel_df and get the Lat/Lng for each city
for index, row in hotel_df.iterrows():
    # get coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    # print response
    results = response['results']
    pprint.pprint(response)
    
    # save the hotel name to dataFrame
    hotel_df.loc[index, "Hotel Name"] = results[0]['name']
   
    
    
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**



In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig